<a href="https://colab.research.google.com/github/Weirdo-001/Brain_Tumor/blob/master/Tumor(87.5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.applications.efficientnet import preprocess_input

# ==========================
# PATHS
# ==========================
DATA_PATH = "/content/drive/MyDrive/datatumor"
TRAIN_DIR = os.path.join(DATA_PATH, "Training")
TEST_DIR  = os.path.join(DATA_PATH, "Testing")

IMG_SIZE = 300
BATCH_SIZE = 32
EPOCHS = 30

# ==========================
# LOAD TRAIN DATA (80-20 SPLIT)
# ==========================
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

# ==========================
# LOAD TEST DATA
# ==========================
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TEST_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_ds.class_names
print("Classes:", class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)

# ==========================
# DATA AUGMENTATION
# ==========================
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.15),
    layers.RandomContrast(0.1),
])



inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

x = data_augmentation(inputs)
x = preprocess_input(x)

base_model = EfficientNetB3(
    include_top=False,
    weights='imagenet',
    input_tensor=x
)

base_model.trainable = False

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

# ==========================
# COMPILE
# ==========================
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.build((None, IMG_SIZE, IMG_SIZE, 3))
model.summary()

# ==========================
# CALLBACKS
# ==========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3)
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)

# ==========================
# TRAIN
# ==========================
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stop, reduce_lr, checkpoint]
)


base_model.trainable = True

for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

# ==========================
# FINAL TEST ACCURACY
# ==========================
test_loss, test_acc = model.evaluate(test_ds)
print("\nFinal Test Accuracy:", test_acc)

# ==========================
# CONFUSION MATRIX
# ==========================
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:\n", cm)

Found 5600 files belonging to 4 classes.
Using 4480 files for training.
Found 5600 files belonging to 4 classes.
Using 1120 files for validation.
Found 1600 files belonging to 4 classes.
Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']
43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 300, 300,  │          0 │ input_layer[0][0] │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 300, 300,  │          0 │ sequential[0][0]  │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 300, 300,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 300, 300,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 301, 301,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 150, 150,  │      1,080 │ stem_conv_pad[0]… │
│                     │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 150, 150,  │        160 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 150, 150,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 150, 150,  │        360 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 150, 150,  │        160 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 150, 150,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 40)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 40)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 10)  │        410 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 40)  │        440 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 150, 150,  │          0 │ block1a_activati

 Total params: 11,184,179 (42.66 MB)

 Trainable params: 397,572 (1.52 MB)

...

140/140 ━━━━━━━━━━━━━━━━━━━━ 45s 322ms/step - accuracy: 0.8601 - loss: 0.3818 - val_accuracy: 0.8964 - val_loss: 0.2742 - learning_rate: 1.0000e-04
Epoch 9/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - accuracy: 0.8653 - loss: 0.3431

140/140 ━━━━━━━━━━━━━━━━━━━━ 45s 321ms/step - accuracy: 0.8653 - loss: 0.3432 - val_accuracy: 0.9045 - val_loss: 0.2561 - learning_rate: 1.0000e-04
Epoch 10/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step - accuracy: 0.8627 - loss: 0.3530

140/140 ━━━━━━━━━━━━━━━━━━━━ 47s 336ms/step - accuracy: 0.8627 - loss: 0.3530 - val_accuracy: 0.9018 - val_loss: 0.2537 - learning_rate: 1.0000e-04
Epoch 11/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 45s 317ms/step - accuracy: 0.8814 - loss: 0.3301 - val_accuracy: 0.9009 - val_loss: 0.2685 - learning_rate: 1.0000e-04
Epoch 12/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.8760 - loss: 0.3209

140/140 ━━━━━━━━━━━━━━━━━━━━ 45s 322ms/step - accuracy: 0.8760 - loss: 0.3208 - val_accuracy: 0.9161 - val_loss: 0.2356 - learning_rate: 1.0000e-04
Epoch 13/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 44s 317ms/step - accuracy: 0.8791 - loss: 0.3345 - val_accuracy: 0.8955 - val_loss: 0.2931 - learning_rate: 1.0000e-04
Epoch 14/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 45s 317ms/step - accuracy: 0.8845 - loss: 0.3122 - val_accuracy: 0.9089 - val_loss: 0.2374 - learning_rate: 1.0000e-04
Epoch 15/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step - accuracy: 0.8854 - loss: 0.2945

140/140 ━━━━━━━━━━━━━━━━━━━━ 83s 324ms/step - accuracy: 0.8853 - loss: 0.2946 - val_accuracy: 0.9170 - val_loss: 0.2337 - learning_rate: 1.0000e-04
Epoch 16/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 44s 314ms/step - accuracy: 0.9029 - loss: 0.2730 - val_accuracy: 0.9045 - val_loss: 0.2534 - learning_rate: 1.0000e-04
Epoch 17/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 44s 317ms/step - accuracy: 0.8957 - loss: 0.2700 - val_accuracy: 0.8964 - val_loss: 0.2745 - learning_rate: 1.0000e-04
Epoch 18/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 44s 316ms/step - accuracy: 0.8922 - loss: 0.2730 - val_accuracy: 0.8982 - val_loss: 0.2744 - learning_rate: 1.0000e-04
Epoch 19/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 44s 316ms/step - accuracy: 0.9021 - loss: 0.2598 - val_accuracy: 0.9036 - val_loss: 0.2473 - learning_rate: 3.0000e-05
Epoch 20/30
140/140 ━━━━━━━━━━━━━━━━━━━━ 44s 316ms/step - accuracy: 0.9104 - loss: 0.2499 - val_accuracy: 0.9071 - val_loss: 0.2435 - learning_rate: 3.0000e-05
Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 80s 414ms/st

In [ ]:
model.save("/content/drive/MyDrive/tumor_model.keras")